In [1]:
import re

regex = r"Mega.*"

test_str = "CharizardMega Charizard X"

matches = re.finditer(regex, test_str, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

Match 1 was found at 9-25: Mega Charizard X


In [1]:
import numpy as np
import pandas as pd

orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
customers = orders.groupby(["CustomerID"])['amount_spent'].sum()

In [4]:
customers.head()

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
Name: amount_spent, dtype: float64

In [6]:
quantile75 = orders.groupby(["CustomerID"])['amount_spent'].sum().quantile(.75)
quantile95 = orders.groupby(["CustomerID"])['amount_spent'].sum().quantile(.95)

In [34]:
# customers < q75 : ordinary
# q75<=customers <q95 :preferred
# customers >=q95 :VIP

customers_status = customers.apply(lambda x: np.where(x<quantile75,'low',np.where(quantile75<=x<quantile95,'preferred','VIP'))).reset_index()
customers_status

,CustomerID,amount_spent
0,12346,VIP
1,12347,preferred
2,12348,preferred
3,12349,preferred
4,12350,low
5,12352,preferred
6,12353,low
7,12354,low
8,12355,low
9,12356,preferred


In [40]:
VIPlist = list(customers_status[customers_status['amount_spent']=="VIP"]["CustomerID"])    
preferredlist=list(customers_status[customers_status['amount_spent']=="preferred"]["CustomerID"]) 
ordinarylist=list(customers_status[customers_status['amount_spent']=="low"]["CustomerID"]) 


In [49]:
#How to identify which country has the most VIP Customers?
orders[orders["CustomerID"].isin(VIPlist)]["Country"].value_counts().reset_index().max()



index      United Kingdom
Country             84185
dtype: object

In [59]:
#Q3: How to identify which country has the most VIP+Preferred Customers combined?

combined = (orders[orders["CustomerID"].isin(VIPlist)]["Country"].value_counts()+orders[orders["CustomerID"].isin(preferredlist)]["Country"].value_counts()).reset_index()
combined.rename(columns={'index':'Country',
                          'Country':'nbcusts'}, 
                 inplace=True)
combined

,Country,nbcusts
0,Australia,1028.0
1,Austria,NaN
2,Belgium,1557.0
3,Canada,NaN
4,Channel Islands,589.0
5,Cyprus,451.0
6,Denmark,217.0
7,EIRE,7238.0
8,Finland,504.0
9,France,6301.0


In [62]:
combined['Country'][combined['nbcusts']==combined['nbcusts'].max()]

26    United Kingdom
Name: Country, dtype: object